In [1]:
import mysql.connector as mysql
import pyodbc as pdbo
from pandas import read_csv
import numpy as np
# Load libraries
import pandas as pd
#import the library for splitting text
import re
from datetime import datetime

In [2]:
server = 'x.database.windows.net'
database = 'x'
username = 'x'
password = '{x}'   
driver= '{ODBC Driver 17 for SQL Server}'

cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
query = ("SELECT SPD.*, S.Location FROM SalesPerDay SPD, Sales S WHERE ItemCode IN (SELECT ItemCodeinItems FROM ReceivedStockHistory) AND S.SalesNo = SPD.SalesNo AND BatchCode IS NULL AND SPD.Date >= '2022-06-01' ORDER BY ItemCode, Date, Time, Qty")
spd = pd.read_sql(query, cnxn)


In [3]:
query = ("SELECT * FROM ReceivedStockHistory WHERE ItemCodeinItems IN (SELECT ItemCode FROM SalesPerDay) AND Status = 'Completed' AND Remaining_Stock > 0 ORDER BY ItemCodeinItems, Date")
rsh = pd.read_sql(query, cnxn)

In [4]:
rsh['DaysUsed']

0        None
1        None
2        None
3        None
4        None
         ... 
43336    None
43337    None
43338    None
43339    None
43340    None
Name: DaysUsed, Length: 43341, dtype: object

In [5]:
rsh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43341 entries, 0 to 43340
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ReceivedID              43341 non-null  object 
 1   Location                43341 non-null  object 
 2   Date                    43341 non-null  object 
 3   Status                  43341 non-null  object 
 4   Receive_Type            43341 non-null  object 
 5   ItemCode                43341 non-null  object 
 6   ItemName                43341 non-null  object 
 7   Qty_Received            43341 non-null  int64  
 8   Qty_Return              43341 non-null  int64  
 9   Qty_Actual_Receive      43341 non-null  int64  
 10  ItemCodeinItems         43341 non-null  object 
 11  ItemIndex               43341 non-null  int64  
 12  SoldOutDate             0 non-null      object 
 13  DaysUsed                0 non-null      object 
 14  BatchCode               43341 non-null

In [6]:
rsh['Date'] = pd.to_datetime(rsh['Date'])
rsh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43341 entries, 0 to 43340
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ReceivedID              43341 non-null  object        
 1   Location                43341 non-null  object        
 2   Date                    43341 non-null  datetime64[ns]
 3   Status                  43341 non-null  object        
 4   Receive_Type            43341 non-null  object        
 5   ItemCode                43341 non-null  object        
 6   ItemName                43341 non-null  object        
 7   Qty_Received            43341 non-null  int64         
 8   Qty_Return              43341 non-null  int64         
 9   Qty_Actual_Receive      43341 non-null  int64         
 10  ItemCodeinItems         43341 non-null  object        
 11  ItemIndex               43341 non-null  int64         
 12  SoldOutDate             0 non-null      object

In [7]:
rsh['Remaining_Stock'] = rsh['Remaining_Stock'].astype(int)
rsh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43341 entries, 0 to 43340
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ReceivedID              43341 non-null  object        
 1   Location                43341 non-null  object        
 2   Date                    43341 non-null  datetime64[ns]
 3   Status                  43341 non-null  object        
 4   Receive_Type            43341 non-null  object        
 5   ItemCode                43341 non-null  object        
 6   ItemName                43341 non-null  object        
 7   Qty_Received            43341 non-null  int64         
 8   Qty_Return              43341 non-null  int64         
 9   Qty_Actual_Receive      43341 non-null  int64         
 10  ItemCodeinItems         43341 non-null  object        
 11  ItemIndex               43341 non-null  int64         
 12  SoldOutDate             0 non-null      object

In [8]:
spd['Date'] = pd.to_datetime(spd['Date'])
spd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5496 entries, 0 to 5495
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ItemID           5496 non-null   object        
 1   SalesNo          5496 non-null   object        
 2   Date             5496 non-null   datetime64[ns]
 3   Time             5496 non-null   object        
 4   Qty              5496 non-null   int64         
 5   NetSoldPrice_$   5492 non-null   float64       
 6   NetSoldProfit_$  5496 non-null   float64       
 7   ItemCode         5496 non-null   object        
 8   BatchCode        0 non-null      object        
 9   CostPerItem_$    5496 non-null   float64       
 10  ItemName         5496 non-null   object        
 11  Profitperitem    5492 non-null   float64       
 12  SalesPrice       5492 non-null   float64       
 13  Location         5496 non-null   object        
dtypes: datetime64[ns](1), float64(5), int64(

In [9]:
count = 0
itemcodecur = ''
for index, row in rsh.iterrows():
    itemcode = str(row['ItemCodeinItems'])
    qty = row['Qty_Received']
    year = row['Date'].year
    month = row['Date'].month
    day = row['Date'].day
    location = row['Location']
    batchcode = row['BatchCode']
    soldoutdate = row['SoldOutDate']
    daysused = row['DaysUsed']
    remainingqty = int(row['Remaining_Stock'])
    latestsalesdaysused = row['LatestSalesDaysUsed']
    if(np.isnan(remainingqty)):
        remainingqty = 0
    qty = remainingqty
    print(qty)
    if (count == 0):
        df2 = spd[spd['ItemCode'] == str(itemcode)]
        df3 = df2[df2['BatchCode'].isnull()]
        count = count + 1
        itemcodecur = str(itemcode)
    if (itemcodecur != str(itemcode)):
        df2 = spd[spd['ItemCode'] == str(itemcode)]
        df3 = df2[df2['BatchCode'].isnull()]
        itemcodecur = str(itemcode)
        print('refresh')
    print(len(df3))
    if(len(df3) != 0 and soldoutdate == None and remainingqty>=0 and (daysused == None or np.isnan(daysused))):
        for index2, row2 in df3.iterrows():
            if(qty != 0):
                if (itemcode == str(row2['ItemCode']) 
                    and row2['Date']>= datetime(year,month,day) 
                    and str(row2['BatchCode']) == "None" # NEED TO 
                    and str(row2['Location']) == location):
                    qtysold = row2['Qty']
                    salesno = row2['SalesNo']
                    itemname = str(row2['ItemName'])
                    print('if')
                    print(row2['BatchCode'])
                    print(itemcode)
                    print(batchcode)
                    print(qty)
                    print(salesno)
                    print(qtysold)
                    cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
                    cursor = cnxn.cursor()
                    if(qtysold - qty <= 0):
                        #sql = ("""WITH UpdateListView AS (SELECT TOP 1 BatchCode FROM SalesPerDay WHERE SalesNo = '%s' AND ItemCode = '%s' AND BatchCode IS NULL) UPDATE SalesPerDay SET BatchCode = '%s' FROM UpdateListView""" % (str(salesno),str(itemcode),str(batchcode)))
                        sql = ("""UPDATE TOP (1) SalesPerDay SET BatchCode = '%s' WHERE SalesNo = '%s' AND ItemCode = '%s' AND BatchCode IS NULL AND Qty = '%s'""" % (str(batchcode),str(salesno),str(itemcode), str(qtysold)))
                        
                        df3.loc[index2,'BatchCode'] = batchcode
                        cursor.execute(sql)
                        cursor.commit() 
                        if(qtysold-qty == 0):
                            date = row2['Date']
                            yearend = row2['Date'].year
                            monthend = row2['Date'].month
                            dayend = row2['Date'].day
                            startday = datetime(year,month,day)
                            endday = datetime(yearend,monthend,dayend)
                            days = (endday-startday).days
                            cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
                            cursor = cnxn.cursor()
                            sql = ("""UPDATE ReceivedStockHistory SET SoldOutDate = '%s' WHERE BatchCode = '%s';""" % (str(date),str(batchcode)))
                            cursor.execute(sql)
                            cursor.commit()
                            cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
                            cursor = cnxn.cursor()
                            sql = ("""UPDATE ReceivedStockHistory SET DaysUsed = '%s' WHERE BatchCode = '%s';""" % (str(days+1),str(batchcode)))
                            cursor.execute(sql)
                            cursor.commit()
                            cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
                            cursor = cnxn.cursor()
                            sql = ("""UPDATE ReceivedStockHistory SET Remaining_Stock = '%s' WHERE BatchCode = '%s';""" % (str(qty-qtysold),str(batchcode)))
                            cursor.execute(sql)
                            cursor.commit()
                        qty = qty-qtysold  
                        
                    else:
                        print('else')
                        print(row2['BatchCode'])
                        print(itemcode)
                        print(batchcode)
                        print(qty)
                        print(salesno)
                        df3.loc[index2, 'Qty'] = (qtysold-qty)
                        ItemID = row2['ItemID']
                        Date = row2['Date']
                        Time = row2['Time']
                        NetSoldPrice = row2['NetSoldPrice_$']
                        NetSoldProfit = row2['NetSoldProfit_$']
                        ItemCode = row2['ItemCode']
                        costperitem = row2['CostPerItem_$']
                        profitperitem = row2['Profitperitem']
                        salesprice = row2['SalesPrice']
                        sql = ("""UPDATE SalesPerDay SET Qty = '%s' WHERE SalesNo = '%s' AND ItemCode = '%s' AND BatchCode IS NULL;""" %(str(qtysold-qty),str(salesno),str(itemcode)))
                        cursor.execute(sql)
                        cursor.commit()
                        cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
                        cursor = cnxn.cursor()
                        sql = ("""UPDATE SalesPerDay SET NetSoldPrice_$ = '%s' WHERE SalesNo = '%s' AND ItemCode = '%s' AND Qty = '%s';""" %(str((NetSoldPrice/qtysold)*(qtysold-qty)),str(salesno),str(itemcode), str(qtysold-qty)))
                        cursor.execute(sql)
                        cursor.commit()
                        cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
                        cursor = cnxn.cursor()
                        sql = ("""UPDATE SalesPerDay SET NetSoldProfit_$ = '%s' WHERE SalesNo = '%s' AND ItemCode = '%s' AND Qty = '%s';""" %(str((NetSoldProfit/qtysold)*(qtysold-qty)),str(salesno),str(itemcode), str(qtysold-qty)))
                        cursor.execute(sql)
                        cursor.commit()
                        cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
                        cursor = cnxn.cursor()
                        sql = ("""SET QUOTED_IDENTIFIER OFF;INSERT INTO SalesPerDay(ItemID, SalesNo, Date, Time, Qty, NetSoldPrice_$, NetSoldProfit_$, ItemCode, BatchCode, CostPerItem_$, ItemName, Profitperitem,SalesPrice) VALUES ('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s',"%s",'%s','%s');SET QUOTED_IDENTIFIER On;""" % (str(ItemID),str(salesno),str(Date),str(Time),str(qty),str((NetSoldPrice/qtysold)*qty),str((NetSoldProfit/qtysold)*qty),str(ItemCode),str(batchcode),str(costperitem),str(itemname),str(profitperitem),str(salesprice)))
                        cursor.execute(sql)
                        cursor.commit()
                        print("inserted")
                        qty = 0
                        date = row2['Date']
                        yearend = row2['Date'].year
                        monthend = row2['Date'].month
                        dayend = row2['Date'].day
                        startday = datetime(year,month,day)
                        endday = datetime(yearend,monthend,dayend)
                        print(startday)
                        print(endday)
                        days = (endday-startday).days
                        cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
                        cursor = cnxn.cursor()
                        sql = ("""UPDATE ReceivedStockHistory SET SoldOutDate = '%s' WHERE BatchCode = '%s';""" % (str(date),str(batchcode)))
                        cursor.execute(sql)
                        cursor.commit()
                        cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
                        cursor = cnxn.cursor()
                        sql = ("""UPDATE ReceivedStockHistory SET DaysUsed = '%s' WHERE BatchCode = '%s';""" % (str(days+1),str(batchcode)))
                        cursor.execute(sql)
                        cursor.commit()
                        cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
                        cursor = cnxn.cursor()
                        sql = ("""UPDATE ReceivedStockHistory SET Remaining_Stock = '%s' WHERE BatchCode = '%s';""" % (str(qty),str(batchcode)))
                        cursor.execute(sql)
                        cursor.commit()
            else:
                rsh.loc[index2,'BatchCode'] = batchcode
                break
        if(row['Qty_Received']> qty):
            cursor = cnxn.cursor()
            sql = ("""UPDATE ReceivedStockHistory SET Remaining_Stock = '%s' WHERE BatchCode = '%s';""" % (str(qty),str(batchcode)))
            cursor.execute(sql)
            cursor.commit()
            print('Latest:'+str(latestsalesdaysused))
            if(latestsalesdaysused != 'None' or ~np.isnan(latestsalesdaysused)):
                cursor = cnxn.cursor()
                sql = ("""UPDATE ReceivedStockHistory SET LatestSalesDaysUsed = NULL WHERE BatchCode = '%s';""" % (str(batchcode)))
                cursor.execute(sql)
                cursor.commit()
                cursor = cnxn.cursor()
                sql = ("""UPDATE ReceivedStockHistory SET LatestSalesDate = NULL WHERE BatchCode = '%s';""" % (str(batchcode)))
                cursor.execute(sql)

                cursor.commit()
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = (";WITH a AS(SELECT Date, BatchCode, ROW_NUMBER() OVER (PARTITION BY BatchCode ORDER BY Date ASC) AS rn FROM SalesPerDay)UPDATE ReceivedStockHistory SET StartSellingDate = a.Date FROM a WHERE rn = 1 AND a.BatchCode IN (SELECT BatchCode FROM ReceivedStockHistory WHERE SoldOutDate IS NOT NULL OR LatestSalesDate IS NOT NULL) AND ReceivedStockHistory.BatchCode = a.BatchCode")
cursor.execute(sql)
cursor.commit()
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = (";WITH a AS(SELECT Date, BatchCode, ROW_NUMBER() OVER (PARTITION BY BatchCode ORDER BY Date DESC) AS rn FROM SalesPerDay)UPDATE ReceivedStockHistory SET LatestSalesDate = a.Date FROM a WHERE rn = 1 AND a.BatchCode IN (SELECT BatchCode FROM ReceivedStockHistory WHERE Remaining_Stock >0 AND DaysUsed IS NULL) AND ReceivedStockHistory.BatchCode = a.BatchCode")
cursor.execute(sql)
cursor.commit()
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = ("UPDATE ReceivedStockHistory SET DaysUsed = DATEDIFF(day, StartSellingDate, SoldOutDate)+1")
cursor.execute(sql)
cursor.commit()
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = ("UPDATE ReceivedStockHistory SET LatestSalesDaysUsed = DATEDIFF(day, StartSellingDate, LatestSalesDate)+1")
cursor.execute(sql)
cursor.commit() 
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = ("UPDATE ReceivedStockHistory SET QuantityPerDay_AllItem = NULL")
cursor.execute(sql)
cursor.commit()
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = ("UPDATE ReceivedStockHistory SET QuantityPerDay_AllItem = CASE WHEN DaysUsed != 0 THEN CAST(CAST(Qty_Received AS FLOAT)/CAST(DaysUsed AS FLOAT)AS FLOAT) WHEN DaysUsed = 0 THEN CAST(Qty_Received AS FLOAT) END WHERE DaysUsed != -1")
cursor.execute(sql)
cursor.commit()
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = ("UPDATE ReceivedStockHistory SET QuantityPerDay_AllItem = CASE WHEN LatestSalesDaysUsed != 0 THEN CAST(CAST((Qty_Received-Remaining_Stock)AS FLOAT)/CAST (LatestSalesDaysUsed AS FLOAT)AS FLOAT) WHEN LatestSalesDaysUsed = 0 THEN CAST ((Qty_Received-Remaining_Stock) AS FLOAT) END WHERE LatestSalesDate IS NOT NULL AND LatestSalesDaysUsed IS NOT NULL")
cursor.execute(sql)
cursor.commit()
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = ("UPDATE ItemnLocation SET QuantityPerDay = NULL")
cursor.execute(sql)
cursor.commit()
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = ("UPDATE ItemnLocation SET QuantityPerDay = a.average FROM (SELECT AVG(RSH.QuantityPerDay_AllItem) AS average, RSH.ItemCodeinItems AS code, RSH.Location AS location FROM ReceivedStockHistory RSH, ItemnLocation I WHERE ItemCodeinItems = I.ItemCode GROUP BY RSH.ItemCodeinItems, RSH.Location) AS a WHERE CONCAT(ItemCode, ItemnLocation.Location) = CONCAT(a.code,a.location)")
cursor.execute(sql)
cursor.commit()
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = ("UPDATE ItemnLocation SET QuantityPerDay = 0 WHERE QuantityPerDay IS NULL")
cursor.execute(sql)
cursor.commit()
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = ("UPDATE SalesPerDay SET NetSoldProfit_$ = ProfitperItem*Qty")
cursor.execute(sql)
cursor.commit()
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = ("UPDATE SalesPerDay SET NetSoldPrice_$ = SalesPrice*Qty")
cursor.execute(sql)
cursor.commit()
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = ("WITH a AS (SELECT sum(Qty)AS qty, BatchCode FROm SalesPerDay WHERE Qty<0 Group BY BatchCode)UPDATE ReceivedStockHistory SET Qty_Received = Qty_Received-a.qty FROm a WHERE ReceivedStockHistory.BatchCode = a.BatchCode")
cursor.execute(sql)
cursor.commit()

48
0
1
0
3
0
40
0
24
refresh
0
48
0
1
0
5
0
3
0
2
refresh
0
2
refresh
0
2
refresh
0
2
0
1
0
3
0
10
refresh
0
4
0
2
0
10
0
5
0
10
0
2
0
25
0
4
0
5
0
13
0
20
refresh
16
20
16
20
16
10
16
5
16
Latest:nan
7
16
Latest:nan
20
16
30
16
10
16
5
16
10
16
30
16
5
16
40
16
5
16
5
16
10
16
40
16
30
16
35
16
5
16
55
16
10
16
5
16
30
16
2
16
Latest:nan
5
16
1
16
Latest:nan
5
16
5
16
10
16
10
16
5
16
35
16
Latest:nan
10
refresh
0
7
0
20
0
10
0
10
0
10
0
10
0
5
0
10
0
25
0
1
0
1
0
15
0
10
0
15
0
16
refresh
0
30
0
6
0
1
0
10
0
5
0
10
0
15
0
5
0
10
0
10
0
13
0
5
0
1
refresh
0
120
refresh
0
2
0
6
0
6
0
2
0
8
0
8
0
2
0
8
0
6
0
8
0
8
0
8
0
10
0
4
0
120
refresh
0
10
0
3
0
9
0
6
0
8
0
8
0
8
0
8
0
8
0
120
refresh
0
8
0
6
0
6
0
8
0
8
0
2
0
2
0
8
0
4
0
8
0
10
0
6
0
120
refresh
0
4
0
9
0
6
0
2
0
2
0
8
0
8
0
8
0
8
0
6
0
120
refresh
0
4
0
4
0
8
0
8
0
8
0
8
0
8
0
6
0
10
0
6
0
120
refresh
0
10
0
6
0
7
0
8
0
2
0
6
0
5
0
5
0
8
0
8
0
12
0
8
0
8
0
6
0
10
0
36
refresh
0
1
0
5
0
6
0
4
0
6
0
6
refresh
0
2
refresh
0
3
refre

12
1
48
refresh
1
3
1
Latest:nan
1
1
Latest:nan
5
1
Latest:nan
8
1
Latest:nan
12
1
120
refresh
4
1
4
Latest:nan
120
4
4
4
Latest:nan
8
4
1
4
Latest:nan
96
refresh
1
1
1
120
1
9
1
Latest:nan
2
1
Latest:nan
12
1
9
1
Latest:nan
7
1
Latest:nan
192
refresh
1
8
1
Latest:nan
48
1
9
1
Latest:nan
3
1
Latest:nan
8
1
Latest:nan
12
1
10
1
Latest:nan
6
refresh
0
4
refresh
0
24
0
1
0
2
0
12
refresh
0
1
0
3
0
1
refresh
0
3
0
24
0
2
0
24
refresh
0
4
0
3
0
5
0
5
0
1
refresh
0
3
0
24
0
6
0
24
refresh
0
6
0
3
0
3
0
4
0
6
0
12
refresh
0
3
0
36
refresh
0
1
0
36
refresh
0
24
refresh
0
6
0
24
refresh
0
2
0
12
0
24
refresh
0
1
0
36
refresh
0
9
refresh
0
6
0
3
refresh
0
30
refresh
0
1
0
30
refresh
0
1
0
4
0
2
0
84
refresh
0
7
0
17
0
13
0
12
0
10
0
9
0
6
0
6
0
5
0
20
0
8
0
5
0
4
0
2
0
2
0
5
0
4
refresh
0
24
refresh
0
2
0
1
0
84
refresh
0
8
0
12
0
21
0
8
0
12
0
6
0
6
0
3
0
20
0
6
0
6
0
6
0
1
0
4
refresh
0
8
0
84
refresh
0
4
0
11
0
18
0
12
0
8
0
6
0
6
0
6
0
5
0
5
0
60
refresh
0
84
refresh
0
16
0
12
0
15
0
8
0
2
0

refresh
0
3
0
3
0
4
0
1
0
4
0
24
refresh
0
5
0
4
0
8
0
1
0
2
0
40
refresh
0
4
0
36
refresh
0
3
0
3
0
10
0
1
0
2
0
36
refresh
0
3
0
11
0
3
0
24
refresh
0
3
0
4
0
5
0
3
0
3
0
30
refresh
0
8
0
2
0
5
0
30
refresh
0
5
0
2
0
8
0
30
refresh
0
2
0
6
0
2
0
10
refresh
0
26
refresh
0
1
0
11
refresh
0
1
0
25
refresh
1
Latest:nan
12
refresh
0
22
refresh
0
15
0
9
0
3
refresh
0
8
0
18
refresh
0
3
0
3
0
2
0
4
0
1
0
18
refresh
0
2
0
5
0
5
0
5
0
2
0
1
refresh
0
1
refresh
0
1
0
1
refresh
0
1
0
1
refresh
0
60
refresh
0
2
0
2
0
4
0
480
refresh
0
480
0
960
0
960
0
120
0
48
0
120
0
20
0
5
0
240
0
8
0
240
0
72
0
24
0
7
0
12
0
10
0
192
0
192
0
48
0
12
0
2
0
4
0
12
0
6
0
4
0
480
refresh
0
960
0
480
0
960
0
48
0
120
0
120
0
1
0
240
0
10
0
240
0
10
0
360
0
48
0
360
0
12
0
240
refresh
0
240
0
480
0
480
0
360
0
17
0
35
0
21
0
11
0
960
0
960
0
360
0
48
0
360
0
12
0
12
0
4
0
2
0
12
0
7
0
30
refresh
0
3
0
3
0
3
0
4
0
4
0
4
0
3
0
1
refresh
0
24
refresh
0
48
0
1
refresh
0
4
0
1
refresh
0
24
refresh
0
4
0
4
0
8
0
4
0
4
0

refresh
0
5
0
5
refresh
0
10
0
10
0
3
refresh
0
6
0
6
0
32
refresh
0
3
0
4
0
1
0
32
refresh
0
3
0
3
0
2
0
48
refresh
0
1
0
48
refresh
0
1
0
2
0
48
refresh
0
1
0
2
0
48
refresh
0
1
0
6
0
2
0
1
0
48
refresh
0
1
0
2
0
40
refresh
0
1
0
40
refresh
0
1
0
2
0
2
0
48
refresh
0
2
0
6
0
6
0
2
0
24
refresh
0
20
0
1
0
24
refresh
0
20
0
2
0
24
refresh
0
20
0
1
0
4
0
24
refresh
0
1
0
3
0
1
0
24
refresh
0
4
0
1
0
4
0
24
refresh
0
1
0
4
0
24
refresh
0
3
0
6
0
6
0
48
refresh
0
2
0
9
0
5
0
6
0
48
refresh
0
5
0
10
0
3
0
6
0
24
refresh
0
3
0
2
0
24
refresh
0
3
0
4
0
24
refresh
0
2
refresh
0
3
refresh
0
2
0
5
refresh
1
1
refresh
0
4
refresh
0
3
0
3
0
6
refresh
0
1
0
6
0
4
0
1
refresh
0
5
0
5
0
96
refresh
0
60
0
200
refresh
0
100
0
96
refresh
0
5
0
60
0
3
refresh
0
144
refresh
0
3
0
7
0
7
0
144
refresh
0
14
0
6
0
14
0
1
refresh
0
6
0
40
refresh
0
2
0
3
0
1
0
1
refresh
0
3
0
40
0
60
refresh
0
2
0
2
0
2
0
2
refresh
0
1
0
40
0
1
0
4
0
40
refresh
0
2
0
2
0
1
0
2
refresh
0
3
refresh
0
1
refresh
0
3
refresh
0
2
r

300
0
7
0
150
0
150
0
75
0
30
0
2
0
6
0
75
0
5
0
10
0
6
0
8
0
300
refresh
0
75
0
15
0
120
0
60
0
300
0
1
0
150
0
1
0
45
0
45
0
7
0
1
0
9
0
6
0
8
0
300
refresh
0
45
0
150
0
150
0
45
0
15
0
45
0
75
0
45
0
150
0
300
0
1
0
150
0
150
0
75
0
75
0
75
0
75
0
30
0
1
0
7
0
4
0
90
0
8
0
8
0
4
0
8
0
8
0
6
0
6
0
300
refresh
2
150
2
90
2
15
2
150
2
90
2
300
2
300
2
225
2
150
2
1
2
Latest:nan
270
2
75
2
75
2
120
2
3
2
Latest:nan
8
2
75
2
11
2
Latest:nan
8
2
8
2
4
2
Latest:nan
8
2
8
2
8
2
5
2
4
refresh
0
6
0
45
0
75
0
2
0
15
0
15
0
120
0
8
0
75
0
8
0
5
0
6
0
8
0
6
0
6
0
1500
refresh
0
1500
0
1500
0
1500
0
1275
0
1500
0
1500
0
450
0
300
0
750
0
1500
0
1500
0
5
0
150
0
1500
0
2250
0
1200
0
1500
0
2250
0
15
0
750
0
2250
0
750
0
6
0
750
0
150
0
15
0
300
0
750
0
300
0
12
0
35
0
450
0
1200
0
13
0
75
0
1425
0
15
0
75
0
750
0
15
0
1500
0
15
0
15
0
600
0
1425
0
15
0
15
0
12
0
9
0
15
0
1
0
15
0
15
0
15
0
15
0
15
0
15
0
15
0
15
0
15
0
15
0
15
0
15
0
1500
refresh
6
1500
6
1800
6
2250
6
15
6
1890
6
1500
6
3000
6
3

...

if
None
ZUGO1084
46--ZUGO1084--2022-06-01
7
2067254
2
if
None
ZUGO1084
46--ZUGO1084--2022-06-01
5
2067393
1
if
None
ZUGO1084
46--ZUGO1084--2022-06-01
4
2067443
1
if
None
ZUGO1084
46--ZUGO1084--2022-06-01
3
2067479
1
if
None
ZUGO1084
46--ZUGO1084--2022-06-01
2
2067518
1
if
None
ZUGO1084
46--ZUGO1084--2022-06-01
1
2067576
1
Latest:nan
3
124
3
124
10
124
if
None
ZUGO1084
561--ZUGO1084--2022-06-03
10
3903711
1
if
None
ZUGO1084
561--ZUGO1084--2022-06-03
9
3903720
1
if
None
ZUGO1084
561--ZUGO1084--2022-06-03
8
2067756
1
if
None
ZUGO1084
561--ZUGO1084--2022-06-03
7
2067760
1
if
None
ZUGO1084
561--ZUGO1084--2022-06-03
6
3903838
2
if
None
ZUGO1084
561--ZUGO1084--2022-06-03
4
2068041
1
if
None
ZUGO1084
561--ZUGO1084--2022-06-03
3
2068061
1
if
None
ZUGO1084
561--ZUGO1084--2022-06-03
2
2068196
1
if
None
ZUGO1084
561--ZUGO1084--2022-06-03
1
2068367
1
Latest:nan
3
124
3
124
12
124
if
None
ZUGO1084
1037--ZUGO1084--2022-06-04
12
2068439
1
if
None
ZUGO1084
1037--ZUGO1084--2022-06-04
11
2068576
1
if
Non

if
None
ZUGO1093
3010--ZUGO1093--2022-06-13
3
2069666
1
if
None
ZUGO1093
3010--ZUGO1093--2022-06-13
2
2070863
1
if
None
ZUGO1093
3010--ZUGO1093--2022-06-13
1
2070983
1
Latest:nan
4
83
4
83
if
None
ZUGO1093
3438--ZUGO1093--2022-06-15
4
2070983
2
if
None
ZUGO1093
3438--ZUGO1093--2022-06-15
2
2071158
1
if
None
ZUGO1093
3438--ZUGO1093--2022-06-15
1
2071158
2
else
None
ZUGO1093
3438--ZUGO1093--2022-06-15
1
2071158
inserted
2022-06-15 00:00:00
2022-06-22 00:00:00
Latest:nan
4
83
4
83
4
83
if
None
ZUGO1093
3881--ZUGO1093--2022-06-17
4
2071158
1
if
None
ZUGO1093
3881--ZUGO1093--2022-06-17
3
2071347
1
if
None
ZUGO1093
3881--ZUGO1093--2022-06-17
2
2071658
2
Latest:nan
4
83
4
83
4
83
4
83
if
None
ZUGO1093
4574--ZUGO1093--2022-06-20
4
2071658
2
if
None
ZUGO1093
4574--ZUGO1093--2022-06-20
2
2072813
1
if
None
ZUGO1093
4574--ZUGO1093--2022-06-20
1
2072813
1
Latest:nan
4
83
4
83
4
83
4
83
if
None
ZUGO1093
5077--ZUGO1093--2022-06-22
4
2073022
1
if
None
ZUGO1093
5077--ZUGO1093--2022-06-22
3
2073022
1
if

if
None
ZUGO1095
2385--ZUGO1095--2022-06-10
2
3903720
1
if
None
ZUGO1095
2385--ZUGO1095--2022-06-10
1
3903726
1
Latest:nan
5
183
if
None
ZUGO1095
2571--ZUGO1095--2022-06-10
5
36019473
1
if
None
ZUGO1095
2571--ZUGO1095--2022-06-10
4
36019525
1
if
None
ZUGO1095
2571--ZUGO1095--2022-06-10
3
36019525
1
if
None
ZUGO1095
2571--ZUGO1095--2022-06-10
2
36019562
1
if
None
ZUGO1095
2571--ZUGO1095--2022-06-10
1
36019562
1
Latest:nan
8
183
if
None
ZUGO1095
2468--ZUGO1095--2022-06-10
8
9030319
2
if
None
ZUGO1095
2468--ZUGO1095--2022-06-10
6
9030446
1
if
None
ZUGO1095
2468--ZUGO1095--2022-06-10
5
9030504
1
if
None
ZUGO1095
2468--ZUGO1095--2022-06-10
4
9030565
1
if
None
ZUGO1095
2468--ZUGO1095--2022-06-10
3
9030565
1
if
None
ZUGO1095
2468--ZUGO1095--2022-06-10
2
9030619
1
if
None
ZUGO1095
2468--ZUGO1095--2022-06-10
1
9030641
1
Latest:nan
9
183
if
None
ZUGO1095
3063--ZUGO1095--2022-06-13
9
9030641
1
if
None
ZUGO1095
3063--ZUGO1095--2022-06-13
8
9030847
1
if
None
ZUGO1095
3063--ZUGO1095--2022-06-13
7
90

if
None
ZUGO1096
5080--ZUGO1096--2022-06-22
3
2071425
1
if
None
ZUGO1096
5080--ZUGO1096--2022-06-22
2
2071508
1
if
None
ZUGO1096
5080--ZUGO1096--2022-06-22
1
2071635
1
Latest:nan
3
16
if
None
ZUGO1096
5179--ZUGO1096--2022-06-22
3
36020506
1
if
None
ZUGO1096
5179--ZUGO1096--2022-06-22
2
36020531
2
Latest:nan
3
16
if
None
ZUGO1096
5878--ZUGO1096--2022-06-25
3
36020853
1
Latest:nan
3
16
if
None
ZUGO1096
5801--ZUGO1096--2022-06-25
3
9032199
1
if
None
ZUGO1096
5801--ZUGO1096--2022-06-25
2
9032399
1
Latest:nan
4
16
if
None
ZUGO1096
5767--ZUGO1096--2022-06-25
4
2072269
1
if
None
ZUGO1096
5767--ZUGO1096--2022-06-25
3
2072380
1
if
None
ZUGO1096
5767--ZUGO1096--2022-06-25
2
2072385
1
if
None
ZUGO1096
5767--ZUGO1096--2022-06-25
1
2072740
1
Latest:nan
3
refresh
24
if
None
ZUGO1099
5180--ZUGO1099--2022-06-22
3
36020506
1
if
None
ZUGO1099
5180--ZUGO1099--2022-06-22
2
36020551
1
if
None
ZUGO1099
5180--ZUGO1099--2022-06-22
1
36021488
1
Latest:nan
4
24
if
None
ZUGO1099
5081--ZUGO1099--2022-06-22
4
2071

OperationalError: ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]TCP Provider: A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond.\r\n (10060) (SQLExecDirectW); [08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure (10060)')